In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from music21 import converter, midi, note, chord, pitch, instrument, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding, Flatten
import pickle
from keras.utils import to_categorical
import pickle

MAESTRO (MIDI and Audio Edited for Synchronous TRacks and Organization) Dataset

In [0]:
data = pd.read_csv("drive/My Drive/maestro-v2.0.0/maestro-v2.0.0.csv")

# Converting midi files to a list of notes, chords and rest sequences

In [0]:
def getComponents(path):
    file = converter.parse(path)
    components = []
    for element in file.recurse():
        if isinstance(element, note.Note):
            components.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            components.append('.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            components.append("Rest")

    return components

Creating a list for notes, chords and rest for 10 songs 

In [24]:
a = []
count = 0
for i in range(10):
  print("{}% Done".format(i*10))
  a.append(getComponents("drive/My Drive/maestro-v2.0.0/" + data["midi_filename"][i]))

0% Done
10% Done
20% Done
30% Done
40% Done
50% Done
60% Done
70% Done
80% Done
90% Done


Stacking the sequences into a single list

In [0]:
notes = [j for element in a for j in element]

# Creating a list of 100 sequences which will be fed to the model for training

In [0]:
def Sequences(notes): 
    seqL = 100
    unique_char = len(set(notes)) 
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - seqL):
        sequence_in = notes[i: i + seqL]
        sequence_out = notes[i + seqL]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, seqL, 1))
    network_input = network_input / float(unique_char)
    network_output = to_categorical(network_output)
    return (network_input, network_output)

# Model

In [0]:
def Normal_LSTM_Model(network_in, unique_char): 
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(unique_char))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

    return model 

# Training

In [0]:
from keras.callbacks import ModelCheckpoint     
def train_network(notes, epochs = 10):
    unique_char = len(set(notes))
    network_input, network_output = Sequences(notes) 
#     model = Normal_LSTM_Model(network_input, unique_char)
    model = load_model('drive/My Drive/Trained_70_Epochs.hdf5')
    print("Model loaded")
    print('Training in progress')
    filepath = 'drive/My Drive/Trained_70_Epochs.hdf5'  
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint]) 
    print('Training completed') 
    return model  

In [29]:
model  = train_network(notes, 5)

Model loaded
Training in progress
Epoch 1/5
27064/27064 [==============================] - 317s 12ms/step - loss: 0.1444 - acc: 0.9641
Epoch 2/5
27064/27064 [==============================] - 315s 12ms/step - loss: 0.1448 - acc: 0.9631
Epoch 3/5
27064/27064 [==============================] - 316s 12ms/step - loss: 0.1451 - acc: 0.9622
Epoch 4/5
27064/27064 [==============================] - 314s 12ms/step - loss: 0.1422 - acc: 0.9643
Epoch 5/5
27064/27064 [==============================] - 315s 12ms/step - loss: 0.1493 - acc: 0.9631
Training completed


# Converting model output into midi notes, chords and rest format

In [0]:
def generate_notes(model, network_input, pitchnames, unique_char):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []
    print('Generating')
    i = 1
    for note_index in range(500):
        if note_index%50 ==0:
          print("{}% Done".format(i*10))
          i += 1
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(unique_char)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = list(pattern)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    print('Done')
    return prediction_output

In [31]:
unique_char = len(set(notes))
network_input, _ = Sequences(notes)
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, network_input, pitchnames, unique_char)

Generating
10% Done
20% Done
30% Done
40% Done
50% Done
60% Done
70% Done
80% Done
90% Done
100% Done
Done


In [0]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == "Rest":
            new_note = note.Rest(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='/content/drive/My Drive/output50Epochs10Songs.mid')
    print('Output Saved as midi file')

In [33]:
create_midi(prediction_output)

Output Saved as midi file
